# Tubería de letalidad sintética dependiente de mutaciones
```
Título original: Mutation dependent synthetic lethal pipeline
Autor: Guangrong Qin
Contacto: gqin@systemsbiology.org
Cuaderno de Jupyter traducido y modificado por Martín Sende Pombo.
Fecha de modificación: 17-02-2024     
Descripción: Este cuaderno se utiliza para responder qué inactivaciones ("knockouts") o silenciamientos ("knockdowns") de genes es probable que muestren sensibilidad a mutaciones en genes específicos.  
```
Citas: Los datos de cribado funcional y los datos ómicos de las líneas celulares proceden de los proyectos Depmap y CCLE del Broad Institute (DepMap Public 20Q3). Para utilizar este cuaderno Jupyter y los datos que se utilizan en él, por favor cite los siguientes artículos:<br/>

Bahar Tercan, Guangrong Qin, Taek-Kyun Kim, Boris Aguilar, Christopher J. Kemp, Nyasha Chambwe, Ilya Shmulevich. SL-Cloud: A Computational Resource to Support Synthetic Lethal Interaction Discovery. BioRxiv 2021.09.18.459450; doi: https://doi.org/10.1101/2021.09.18.459450

Para esta versión de DepMap:
DepMap, Broad (2020): DepMap 20Q3 Public. figshare. Dataset doi:10.6084/m9.figshare.11791698.v2.

Para los conjuntos de datos de CRISPR:
Robin M. Meyers, Jordan G. Bryan, James M. McFarland, Barbara A. Weir, ... David E. Root, William C. Hahn, Aviad Tsherniak. Computational correction of copy number effect improves specificity of CRISPR-Cas9 essentiality screens in cancer cells. Nature Genetics 2017 October 49:1779–1784. doi:10.1038/ng.3984. PMID: 29083409

Dempster, J. M., Rossen, J., Kazachkova, M., Pan, J., Kugener, G., Root, D. E., & Tsherniak, A. (2019). Extracting Biological Insights from the Project Achilles Genome-Scale CRISPR Screens in Cancer Cell Lines. BioRxiv, 720243.

Para conjuntos de datos ómicos:
Mahmoud Ghandi, Franklin W. Huang, Judit Jané-Valbuena, Gregory V. Kryukov, ... Todd R. Golub, Levi A. Garraway & William R. Sellers. 2019. Next-generation characterization of the Cancer Cell Line Encyclopedia. Nature 569, 503–508 (2019).PMID: 31068700


In [1]:
# Comprobación de las bibliotecas requeridas.
try:
    from google.cloud import bigquery
    print("El módulo 'google-cloud-bigquery' está instalado.")
except ModuleNotFoundError:
    !pip3 install google-cloud-bigquery
    from google.cloud import bigquery

try:
    import ipywidgets as widgets
    print("El módulo 'ipywidgets' está instalado.")
except ModuleNotFoundError:
    !pip3 install ipywidgets
    import ipywidgets as widgets

try:
    import pyarrow
    print("El módulo 'pyarrow' está instalado.")
except ModuleNotFoundError:
    !pip3 install pyarrow
    import pyarrow

try:
    import pandas as pd
    print("El módulo 'pandas' está instalado.")
except ModuleNotFoundError:
    !pip3 install pandas
    import pandas as pd

try:
    import numpy as np
    print("El módulo 'numpy' está instalado.")
except ModuleNotFoundError:
    !pip3 install numpy
    import numpy as np

try:
    from scipy import stats    
    print("El módulo 'scipy' está instalado.")
except ModuleNotFoundError:
    !pip3 install scipy
    from scipy import stats    

try:
    import statsmodels.stats.multitest as multi   
    print("El módulo 'statsmodels' está instalado.")
except ModuleNotFoundError:
    !pip3 install statsmodels
    import statsmodels.stats.multitest as multi

!pip3 install -i https://test.pypi.org/simple/ MDSLP==0.4
from MDSLP import MDSLP


  Obtaining dependency information for google-cloud-bigquery from https://files.pythonhosted.org/packages/49/dc/df04020251473cacbb0001bc8c1c065ea46d6ce4d4dd218b3bfa246bdb5a/google_cloud_bigquery-3.17.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-api-core!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.0,<3.0.0dev,>=1.31.5 from https://files.pythonhosted.org/packages/0f/87/373ab788a4682adc1a6900e54d54c750b7bd4be456d75b8bf64eccc23ef9/google_api_core-2.17.1-py3-none-any.whl.metadata
  Obtaining dependency information for google-cloud-core<3.0.0dev,>=1.6.0 from https://files.pythonhosted.org/packages/5e/0f/2e2061e3fbcb9d535d5da3f58cc8de4947df1786fe6a1355960feb05a681/google_cloud_core-2.4.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-resumable-media<3.0dev,>=0.6.0 from https://files.pythonhosted.org/packages/b2/c6/1202ef64a9336d846f713107dac1c7a0b016cb3840ca3d5615c7005a23d1/google_resumable_media-2.7.0-py2.py3-none-any.whl.metadata
  Obtaining de

## Autentificación de Google
Ejecutar las celdas de BigQuery en este cuaderno requiere un proyecto de Google Cloud y las instrucciones para crear uno pueden encontrarse en la [documentación de Google](https://cloud.google.com/resource-manager/docs/creating-managing-projects#console). Además, la instancia debe estar autorizada para facturar las consultas del proyecto.
Para más información sobre cómo iniciarse en la nube, consulte ['Quick Start Guide to ISB-CGC'](https://isb-cancer-genomics-cloud.readthedocs.io/en/latest/sections/HowToGetStartedonISB-CGC.html). Asimismo, puede localizar métodos de autenticación alternativos en la [documentación de Google](https://googleapis.dev/python/google-api-core/latest/auth.html).

### * Si está ejecutando este cuaderno en un entorno de máquina virtual de Google (Compute Engine, App Engine, Cloud Run, Cloud Functions): 
La autenticación debería "simplemente funcionar".

### * Si la ejecución se realiza en un entorno local: 
La forma más sencilla de autenticarse es emplear el SDK de Google Cloud.

In [4]:
# Los usuarios necesitan ejecutar el siguiente bloque de código en su máquina local o a través del cuaderno.
# Asegúrese de instalar la nube de google en el entorno local. Para más detalles sobre la instalación de gcloud, consulte el soporte en https://cloud.google.com/sdk/docs/install

#!pip3 install --upgrade google-auth
!gcloud auth application-default login

"gcloud" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


###

In [ ]:
%load_ext google.cloud.bigquery

## Establecer la entrada 1 del usuario:
1. Data_source: la fuente de datos deseada, ya sea ARNhc o CRISPR. Tipo de dato: cadena de caracteres ("string").
2. Genes mutados que deben investigarse: una lista de un solo gen o de varios genes. Tipo de dato: lista.

In [ ]:
# Los usuarios deben autenticarse con su proyecto en la nube de Google para consultar los datos de las tablas de BigQuery. 
project_id='crucial-minutia-407312' # Sustituye este identificador por el del proyecto de Google del usuario
client = bigquery.Client(project_id)

# Entrada del usuario; La pregunta en lenguaje natural que planteamos aquí es qué gen muestra una interacción letal sintética con el gen diana.
Data_source = "shRNA" # Sólo hay dos opciones disponibles, "shRNA" o "Crispr", tipo de dato: string

#Gene_list = ['BRCA2'] # Tipo de dato: lista de símbolos de genes

In [ ]:
# Para utilizar su propia lista de genes, cópiela en la carpeta "Listas_genes" y reemplace el nombre del archivo_actual ("Genes_AmiGO2_ruta_biosintesis_NAD.txt") por el del suyo.
# Función para cambiar el formato de la lista de caracteres
def cambiar_formato_lista(lista):
    # Dividir la lista en líneas y eliminar líneas vacías
    lineas = lista.split('\n')
    lineas = [linea.strip() for linea in lineas if linea.strip()]

    # Devolver la lista en el formato deseado
    return lineas

# Ruta del archivo que contiene la lista en el formato actual
archivo_actual = 'Listas_genes/Genes_AmiGO2_ruta_biosintesis_NAD.txt'

# Lee la lista en el formato actual desde el archivo
lista_actual = ""
with open(archivo_actual, 'r') as archivo:
    lista_actual = archivo.read()

# Cambia el formato de la lista
Gene_list = cambiar_formato_lista(lista_actual)

In [ ]:
# Opcionalmente puede ejecutarse este bloque de código, para verificar que la lista de genes se ha almacenado correctamente y que tiene el formato adecuado.
# Concretamente, imprime la lista ya con el formato deseado.
print (Gene_list)

# Si el resultado es el esperado, ['GENE1', 'GENE2', ... 'GENEn'], entoces puede proceder con el siguiente paso.

## Fijar la entrada 2 del usuario:
Tipos de tumores considerados. Los usuarios pueden seleccionar uno o varios tipos de tumores para su análisis.

In [ ]:
query = ''' 
SELECT DepMap_ID, primary_disease,TCGA_subtype
FROM `syntheticlethality.DepMap_public_20Q3.sample_info_Depmap_withTCGA_labels` 
'''
sample_info = client.query(query).result().to_dataframe()

pancancer_cls = sample_info.loc[~sample_info['primary_disease'].isin(['Non-Cancerous','Unknown','Engineered','Immortalized'])]
pancancer_cls = pancancer_cls.loc[~(pancancer_cls['primary_disease'].isna())]

TCGA_list = [x for x in list(set(pancancer_cls['primary_disease'])) if x == x]

Not_none_values = filter(None.__ne__, TCGA_list)
TCGA_list = list(Not_none_values)

tumor_type = widgets.SelectMultiple(
    options=['pancancer'] + TCGA_list  ,
    value=[],
    description='Tumor type',
    disabled=False
)
display(tumor_type)

#### Obtenga datos de mutación de la CCLE, efectos del "knockout" de genes con CRISPR del Depmap y, datos de dependencia génica de genes silenciados con ARNhc de DEMETER2 v6.
La versión 20Q3 de Depmap se utilizará en el siguiente análisis.

In [ ]:
# Consulta de recursos de datos para análisis posteriores
client = bigquery.Client(project_id)

# Asignación de ID entre la anotación de la CCLE y los símbolos genéticos de entrada
id_mapping, Gene_list_matched = MDSLP.GeneSymbol_standardization(Gene_list, project_id)

# Obtener los datos de mutación, los datos de ARNhc o el conjunto de datos CRISPR
Mut_mat = MDSLP.get_ccle_mutation_data(project_id) # Obtiene la tabla de mutaciones de las líneas celulares de la CCLE (versión: Depmap 20Q3)

if Data_source == "shRNA" :
    Demeter_data = MDSLP.get_demeter_shRNA_data(project_id) # Obtiene los efectos del silenciamiento génico basado en ARNhc del proyecto Depmap (DEMETER2)
elif Data_source == "Crispr": 
    Depmap_matrix = MDSLP.get_depmap_crispr_data(project_id) # Obtiene los efectos del bloqueo génico basado en CRISPR del proyecto Depmap (versión: Depmap 20Q3)
else:
    print("Data_source sólo tiene dos opciones: shRNA or Crispr")


##### 
Usted está esperando ver el mensaje anterior de la siguiente manera:

Unnamed: 0 <br/>
AZ521_STOMACH<br/>
GISTT1_GASTROINTESTINAL_TRACT<br/>
MB157_BREAST<br/>
SW527_BREAST<br/>
<br/>
Significa que estas líneas celulares no se incluyen en el análisis, ya que muestran anotaciones no coincidentes de diferentes conjuntos de datos.


## Seleccione el conjunto de datos de ARNhc o CRISPR para inferir pares de letalidad sintética para genes mutados.

In [ ]:
if Data_source == "shRNA":
    result = MDSLP.Mutational_based_SL_pipeline(list(tumor_type.value), Gene_list_matched, Mut_mat, Demeter_data, Data_source,project_id)
    if result.shape[0] > 0:
        result_sig = result.loc[result['FDR_all_exp'] < 0.05]
        result_sig = result_sig.loc[result_sig['ES']<0] # ES < 0 representa los pares SL
    else:
        result_sig = pd.DataFrame()
        
elif  Data_source == "Crispr":
    result = MDSLP.Mutational_based_SL_pipeline(list(tumor_type.value), Gene_list_matched, Mut_mat, Depmap_matrix, Data_source,project_id)
    if result.shape[0] > 0:
        result_sig = result.loc[result['FDR_all_exp'] < 0.05]
        result_sig = result_sig.loc[result_sig['ES'] < 0]  # ES < 0 representa los pares SL
    else:
        result_sig = pd.DataFrame()

else:
    print("El tipo de fuente de datos no existe.")

In [ ]:
result_sig.sort_values(by = ['FDR_all_exp'])

In [5]:
result_sig.to_csv("result_sig.csv")
result_sig.to_excel("result_sig.xlsx")

NameError: name 'result_sig' is not defined

**Interpretación de resultados**

La tabla result_sig contiene los pares de genes letales sintéticos predichos a partir de este "pipeline".<br/>
**Anotación en la tabla:**

Gene_mut: genes mutados.<br/>
Gene_kd: gen "knockdown" o "knockout". <br/>
Mutated_samples: número de líneas celulares mutadas en el tipo de tumor seleccionado.<br/>
pvalue: valor-p resultante de la prueba t.<br/>
ES: tamaño del efecto de los genes entre el grupo mutado y el grupo de tipo salvaje.<br/>
FDR_all_exp: FDR para el p-valor de todos los análisis.<br/>
FDR_by_gene: FDR para el valor P por cada mutación de un gen.<br/>
Tumor_type: tipos de tumores en análisis.